In [ ]:
from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig
import os


eval_cfg = EvalConfig(
    inception_score=True,
    fid=True,
    kid=True,
    prc=False,
    fid_infinity=False,
    is_infinity=False,
    clean_fid=False,
    clean_kid=False,
    clean_fid_batch_size=32,
    clean_kid_batch_size=32,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

result_dict = platform_manager.calc_metrics()
result_dict.print()
result_dict.write_to_json(file="is_fid_infty.json")

In [1]:
from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig
import os



eval_cfg = EvalConfig(
    c2st_knn=True,
    c2st_k=1,
    c2st_k_adaptive=True,
    c2st_num_samples=25000,
    c2st_folds=5,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True,
    num_worker=16,
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

result_dict = platform_manager.calc_metrics()
result_dict.print()
result_dict.write_to_json(file="c2st_2.json")

[INFO]: Real images source found, name:CelebA64 (Original)
[INFO]: 4 different generator sources found. Names:
cifar-10
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP
[INFO]: Comparison real-to-real (True)
[START]: Calculating Metrics for cifar-10
[INFO]: Start Calculation C2ST-1NN, Source = cifar-10
[INFO]: Feature Extractor used: InceptionV3
[INFO]: Downsampling real data
[INFO]: Compute real features


[INFO]: Feature Extraction: 100%|##########| 782/782 [01:19<00:00,  9.80it/s]


[INFO]: Downsampling generated data
[INFO]: Compute generated features


[INFO]: KNN Cross Valiation: 100%|##########| 5/5 [00:26<00:00,  5.36s/it]


[INFO]: C2ST-1NN finished
[FINISHED]: Calculating Metrics for cifar-10
{'C2ST Adaptive KNN Accuracy': 0.98712, 'C2ST Adaptive KNN Normalized': 0.025760000000000005}
[START]: Calculating Metrics for diffusionStyleGAN2
[INFO]: Start Calculation C2ST-1NN, Source = diffusionStyleGAN2
[INFO]: Feature Extractor used: InceptionV3
[INFO]: Used cached real features
[INFO]: Downsampling generated data
[INFO]: Compute generated features


...

[INFO]: C2ST-1NN finished
[FINISHED]: Calculating Metrics for diffusionVAE
{'C2ST Adaptive KNN Accuracy': 0.90848, 'C2ST Adaptive KNN Normalized': 0.1830400000000001}
[START]: Calculating Metrics for guidedDiffusion_IP
[INFO]: Start Calculation C2ST-1NN, Source = guidedDiffusion_IP
[INFO]: Feature Extractor used: InceptionV3
[INFO]: Used cached real features
[INFO]: Downsampling generated data
[INFO]: Compute generated features


[INFO]: KNN Cross Valiation: 100%|##########| 5/5 [00:27<00:00,  5.44s/it]


[INFO]: C2ST-1NN finished
[FINISHED]: Calculating Metrics for guidedDiffusion_IP
{'C2ST Adaptive KNN Accuracy': 0.5917000000000001, 'C2ST Adaptive KNN Normalized': 0.8165999999999998}
[START]: Calculating Metrics for CelebA64 (Original)
[INFO]: Start Calculation C2ST-1NN, Source = CelebA64 (Original)
[INFO]: Feature Extractor used: InceptionV3
[INFO]: Used cached real features
[INFO]: Compute generated features


[INFO]: KNN Cross Valiation: 100%|##########| 5/5 [00:26<00:00,  5.29s/it]

[INFO]: C2ST-1NN finished
[FINISHED]: Calculating Metrics for CelebA64 (Original)
{'C2ST Adaptive KNN Accuracy': 0.5, 'C2ST Adaptive KNN Normalized': 1.0}
------------------------------------
Generator Name: cifar-10
------------------------------------
C2ST Adaptive KNN Accuracy: 0.98712
C2ST Adaptive KNN Normalized: 0.025760000000000005
------------------------------------
Generator Name: diffusionStyleGAN2
------------------------------------
C2ST Adaptive KNN Accuracy: 0.72324
C2ST Adaptive KNN Normalized: 0.55352
------------------------------------
Generator Name: diffusionVAE
------------------------------------
C2ST Adaptive KNN Accuracy: 0.90848
C2ST Adaptive KNN Normalized: 0.1830400000000001
------------------------------------
Generator Name: guidedDiffusion_IP
------------------------------------
C2ST Adaptive KNN Accuracy: 0.5917000000000001
C2ST Adaptive KNN Normalized: 0.8165999999999998
------------------------------------
Generator Name: CelebA64 (Original)
----------

In [2]:
from framework.platform import ResultDict

results = ResultDict({})
results.read_from_json("all.json")
results.print(round_scores=True)

------------------------------------
Generator Name: cifar-10
------------------------------------
Inception Score Mean: 11.189
Inception Score Std: 0.115
Frechet Inception Distance: 204.998
Kernel Inception Distance Mean: 0.186
Kernel Inception Distance Std: 0.003
Precision: 0.085
Recall: 0.001
F1 Score: 0.002
FID Infinity (Approx.): 225.647
IS Infinity (Approx.): 9.475
Clean FID: 211.253
Clean KID: 0.195
LS: 0.557
C2ST Adaptive KNN Accuracy: 0.987
C2ST Adaptive KNN Normalized: 0.026
------------------------------------
Generator Name: diffusionStyleGAN2
------------------------------------
Inception Score Mean: 3.117
Inception Score Std: 0.028
Frechet Inception Distance: 11.367
Kernel Inception Distance Mean: 0.011
Kernel Inception Distance Std: 0.001
Precision: 0.613
Recall: 0.362
F1 Score: 0.455
FID Infinity (Approx.): 8.723
IS Infinity (Approx.): 3.16
Clean FID: 11.949
Clean KID: 0.011
LS: 0.593
C2ST Adaptive KNN Accuracy: 0.723
C2ST Adaptive KNN Normalized: 0.554
----------------